# Modelo de Predicción de Multas y Sanciones

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',None)

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [5]:
os.chdir('/gdrive/MyDrive/Colab Notebooks/Proyecto Riesgo Corrupción')

In [6]:
!ls

consulta.xlsx				 obra
Consulta.xlsx				 obra_hospitales
data_engineering_multas_sanciones.ipynb  pae
filtrar_contratos_secop.ipynb		 secop_i_descarga_2020
Indicadores-BanderasRojas.ipynb		 secop_ii
interventoria.xlsx			 secop_i_match_multas
multas


In [ ]:
multas = pd.read_csv('multas/multas_sanciones_secop_i.csv')
multas.head()

,Nombre Entidad,NIT Entidad,Nivel,Orden,Municipio,Numero de Resolucion,Documento Contratista,Nombre Contratista,Numero de Contrato,Valor Sancion,Fecha de Publicacion,Ruta de Proceso
0,AGENCIA NACIONAL DE INFRAESTRUCTURA ANI,830125996,NACIONAL,NACIONAL DESCENTRALIZADO,Buenaventura,RESOLUCION 822-2018 Y RESOLUCION 1284-2018,9002251332,SOCIEDAD FERROCARRIL DEL PACIFICO SAS,CONTRATO DE CONCESION No 09CONP981998,0.000000e+00,07/18/2018 12:00:00 AM,https://www.contratos.gov.co/consultas/detalle...
1,BOGOTÁ DC IDU,No Definido,TERRITORIAL,DISTRITO CAPITAL,Bogotá D.C.,14257,9002592606,CONSORCIO PRO 3,IDU932008,1.044120e+09,05/08/2014 12:00:00 AM,https://www.contratos.gov.co/consultas/detalle...
2,CASANARE GOBERNACIÓN,892099216,TERRITORIAL,TERRITORIAL DEPARTAMENTAL CENTRALIZADO,Villanueva,RESOLUCION 0064-2019 Y RESOLUCION 0070-2019,9008530116,CONSORCIO EDUCAR 2015,CTO 13212015,3.914187e+07,03/06/2019 12:00:00 AM,https://www.contratos.gov.co/consultas/detalle...
3,SANTANDER METROLINEA SA,830507387,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,Bucaramanga,035,900186817,METROCINCO PLUS SA,CONTRATO DE CONCESION UNO MLP0032007,9.856000e+05,02/05/2014 12:00:00 AM,https://www.contratos.gov.co/consultas/detalle...
4,SUPERINTENDENCIA NACIONAL DE SALUD SUPERSALUD,860062187,NACIONAL,NACIONAL DESCENTRALIZADO,Bogotá D.C.,2863,9008735825,CONSORCIO AUDIESPECIALES 2015,106 DE 2015,1.639915e+07,12/30/2015 12:00:00 AM,https://www.contratos.gov.co/consultas/detalle...


In [ ]:
"""
# Hacer el match entre secop y multas y sanciones
#!pip install fuzzymatcher
import fuzzymatcher

left_on = ["Nombre Entidad", "Nombre Contratista", "Numero de Contrato"]
right_on = ['Nombre Entidad','Nom Razon Social Contratista','Numero de Contrato']
bases_secop = ['secop_i_procesos_2014.gz',
               'secop_i_procesos_2015.gz',
               'secop_i_procesos_2016.gz',
               'secop_i_procesos_2017.gz',
               'secop_i_procesos_2018.gz',
               'secop_i_procesos_2019.gz',
               #'secop_i_procesos_2020.gz',
               #'secop_i_procesos_2021.gz',
               #'secop_i_procesos_2022.gz'
               ]

for i in bases_secop:
  # 1. Leer base de datos de secop
  secop = pd.read_csv(i, usecols=['UID','Nombre Entidad','Numero de Contrato','Modalidad de Contratacion','Tipo De Contrato','Nom Razon Social Contratista'])

  print(i + ' -> Cargado')
  # 2. Seleccionar las modalidad de contratación de interes
  #secop['Modalidad de Contratacion'].value_counts()

 #  modalidad = ['Licitación Pública',
#                'Selección Abreviada servicios de Salud',
#                'Selección Abreviada del literal h del numeral 2 del artículo 2 de la Ley 1150 de 2007',
#                'Selección Abreviada de Menor Cuantía (Ley 1150 de 2007)',
#                'Subasta']

  modalidad = [
             'Contratación Directa (Ley 1150 de 2007)',
             'Régimen Especial'
            ]
  tipo_contrato = ['Suministro','Compraventa','Obra']

  # 3. Filtrar secop segun la modadlide de interes
  secop_merge = secop.loc[(secop['Modalidad de Contratacion'].isin(modalidad)) & 
                          (secop['Tipo De Contrato'].isin(tipo_contrato)),
                          ['UID','Nombre Entidad','Numero de Contrato','Nom Razon Social Contratista']].copy()

  # 4. Realizar el match con la base de multas y sanciones
  matched_results = fuzzymatcher.fuzzy_left_join(multas[left_on],
                                                 secop_merge,
                                                 left_on,
                                                 right_on)
  
  print(i + ' -> Match')
  
  # 5. Copiar los resultados en excel
  file_name = i[:-3] + '_match_multas_modalidad2.xlsx'
  matched_results.loc[matched_results.best_match_score > 0.5].sort_values(by='best_match_score', ascending=False).to_excel(file_name, index=False)
  print(i + ' -> Match copiado')
  """

In [ ]:
match_archivos = ['secop_i_procesos_2014_match_multas.xlsx',
                  'secop_i_procesos_2015_match_multas.xlsx',
                  'secop_i_procesos_2016_match_multas.xlsx',
                  'secop_i_procesos_2017_match_multas.xlsx',
                  'secop_i_procesos_2018_match_multas.xlsx',
                  'secop_i_procesos_2019_match_multas.xlsx']

for i in range(len(match_archivos)):
  if i == 0:
    secop_i_match = pd.read_excel('secop_i_match_multas/' + match_archivos[i])
  else:
    temp = pd.read_excel('secop_i_match_multas/' + match_archivos[i])
    secop_i_match = pd.concat([secop_i_match, temp])

In [ ]:
secop_i_match['Match'].value_counts()

no         454
si         220
revisar     69
Name: Match, dtype: int64

In [ ]:
# Contratos unicos
secop_i_match.loc[secop_i_match['Match'] == 'si']['UID'].nunique()

127

In [ ]:
"""
# Crear una base de datos con contratos de interes

#bases_secop = ['secop_i_procesos_2014.gz',
#               'secop_i_procesos_2015.gz',
#               'secop_i_procesos_2016.gz',
#               'secop_i_procesos_2017.gz',
#               'secop_i_procesos_2018.gz',
#               'secop_i_procesos_2019.gz']

bases_secop = ['secop_i_2012.gz',
               'secop_i_2013.gz'
#               'secop_i_2014.gz',
#               'secop_i_2015.gz',
#               'secop_i_2016.gz',
#               'secop_i_2017.gz',
#               'secop_i_2018.gz',
#               'secop_i_2019.gz'
               ]

#modalidad = ['Licitación Pública',
#                'Selección Abreviada servicios de Salud',
#                'Selección Abreviada del literal h del numeral 2 del artículo 2 de la Ley 1150 de 2007',
#                'Selección Abreviada de Menor Cuantía (Ley 1150 de 2007)',
#                'Subasta']

for i in range(len(bases_secop)):
  
  if i == 0:
    secop_i = pd.read_csv('secop_i_descarga_2020/' + bases_secop[i])
    secop_i = secop_i.loc[secop_i['Tipo de Proceso'].isin(modalidad)]
  else:
    temp =  pd.read_csv('secop_i_descarga_2020/' + bases_secop[i])
    temp = temp.loc[temp['Tipo de Proceso'].isin(modalidad)]
    secop_i = pd.concat([secop_i, temp])

secop_i.to_csv('secop_i_descarga_2020/secop_i_licitacion_selabre_subasta.gz', index=False)
"""

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
"""
# Extraer contratos de obra

bases_secop = [
               'secop_i_2012.gz',
               'secop_i_2013.gz',
               'secop_i_2014.gz',
               'secop_i_2015.gz',
               'secop_i_2016.gz',
               'secop_i_2017.gz',
               'secop_i_2018.gz',
               'secop_i_2019.gz'
               ]

for i in bases_secop:
  if i == 'secop_i_2012.gz':
    secop_obra = pd.read_csv('secop_i_descarga_2020/' + i)
    secop_obra = secop_obra[(secop_obra['Tipo de Contrato'] == 'Obra')]
    print("listo " + i)
  else:
    temp = pd.read_csv('secop_i_descarga_2020/' + i)
    temp = temp[(temp['Tipo de Contrato'] == 'Obra')]
    secop_obra = pd.concat([secop_obra, temp])
    print("listo " + i)

secop_obra['Multa'] = np.int16(secop_obra['UID'].isin(secop_i_match.loc[secop_i_match['Match'] == 'si']['UID'].unique()))
secop_obra.to_csv('secop_i_descarga_2020/secop_i_obra.gz', index=False)
"""

listo secop_i_2012.gz
listo secop_i_2013.gz
listo secop_i_2014.gz
listo secop_i_2015.gz
listo secop_i_2016.gz
listo secop_i_2017.gz


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


listo secop_i_2018.gz
listo secop_i_2019.gz


In [7]:
# Extraer contratos de PAE

bases_secop = [
               #'secop_i_2012.gz',
               #'secop_i_2013.gz',
               'secop_i_2014.gz',
               'secop_i_2015.gz',
               'secop_i_2016.gz',
               #'secop_i_2017.gz',
               #'secop_i_2018.gz',
               #'secop_i_2019.gz'
               ]

for i in bases_secop:
  if i == 'secop_i_2014.gz':
    secop_pae = pd.read_csv('secop_i_descarga_2020/' + i)
    secop_pae['Detalle del Objeto a Contratar'] = secop_pae['Detalle del Objeto a Contratar'].str.lower()
    secop_pae = secop_pae.loc[(secop_pae['Detalle del Objeto a Contratar'].str.contains("pae|programa de alimentación escolar|programa de alimentacion escolar|alimentación escolar",na=0))].copy()
    print("listo " + i)
  else:
    temp = pd.read_csv('secop_i_descarga_2020/' + i)
    temp['Detalle del Objeto a Contratar'] = temp['Detalle del Objeto a Contratar'].str.lower()
    temp = temp.loc[(temp['Detalle del Objeto a Contratar'].str.contains("pae|programa de alimentación escolar|programa de alimentacion escolar|alimentación escolar",na=0))].copy()
    secop_pae = pd.concat([secop_pae, temp])
    del temp
    print("listo " + i)

#secop_obra['Multa'] = np.int16(secop_obra['UID'].isin(secop_i_match.loc[secop_i_match['Match'] == 'si']['UID'].unique()))
secop_pae.to_csv('secop_i_descarga_2020/secop_i_pae_1.gz', index=False)

listo secop_i_2014.gz
listo secop_i_2015.gz
listo secop_i_2016.gz


In [1]:
secop_pae.shape

NameError: ignored

In [ ]:
secop_obra = pd.read_csv('secop_i_descarga_2020/secop_i_obra.gz')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
secop_obra.shape

(226154, 72)

In [ ]:
secop_obra['Tipo de Proceso'].value_counts()

Contratación Mínima Cuantía                                                              89045
Selección Abreviada de Menor Cuantía (Ley 1150 de 2007)                                  45559
Régimen Especial                                                                         32339
Licitación Pública                                                                       28959
Contratación Directa (Ley 1150 de 2007)                                                  23565
Licitación obra pública                                                                   5229
Contratos y convenios con más de dos partes                                                772
Concurso de Méritos Abierto                                                                386
Selección Abreviada del literal h del numeral 2 del artículo 2 de la Ley 1150 de 2007      154
Subasta                                                                                    106
Selección Abreviada servicios de Salud            

In [ ]:
#secop_i = pd.read_csv('secop_i_descarga_2020/secop_i_licitacion_selabre_subasta.gz')
#secop_i.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(172383, 71)

In [ ]:
secop_obra.head()

,UID,Anno Cargue SECOP,Anno Firma del Contrato,Nivel Entidad,Orden Entidad,Nombre de la Entidad,NIT de la Entidad,Código de la Entidad,ID Tipo de Proceso,Tipo de Proceso,Estado del Proceso,Causal de Otras Formas de Contratacion Directa,ID Regimen de Contratacion,Regimen de Contratacion,ID Objeto a Contratar,Objeto a Contratar,Detalle del Objeto a Contratar,Tipo de Contrato,Municipio Obtencion,Municipio Entrega,Municipios Ejecucion,Fecha de Cargue en el SECOP,Numero de Constancia,Numero de Proceso,Numero del Contrato,Cuantia Proceso,ID Grupo,Nombre Grupo,ID Familia,Nombre Familia,ID Clase,Nombre Clase,ID Ajudicacion,Tipo Identifi del Contratista,Identificacion del Contratista,Nom Raz Social Contratista,Dpto y Muni Contratista,Tipo Doc Representante Legal,Identific del Represen Legal,Nombre del Represen Legal,Fecha de Firma del Contrato,Fecha Ini Ejec Contrato,Plazo de Ejec del Contrato,Rango de Ejec del Contrato,Tiempo Adiciones en Dias,Tiempo Adiciones en Meses,Fecha Fin Ejec Contrato,Compromiso Presupuestal,Cuantia Contrato,Valor Total de Adiciones,Valor Contrato con Adiciones,Objeto del Contrato a la Firma,ID Origen de los Recursos,Origen de los Recursos,Codigo BPIN,Proponentes Seleccionados,Calificacion Definitiva,ID Sub Unidad Ejecutora,Nombre Sub Unidad Ejecutora,Ruta Proceso en SECOP I,Moneda,EsPostConflicto,Marcacion Adiciones,Posicion Rubro,Nombre Rubro,Valor Rubro,Sexo RepLegal Entidad,Pilar Acuerdo Paz,Punto Acuerdo Paz,Municipio Entidad,Departamento Entidad,Multa
0,12-13-1295848-1219669,2012,2012.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,BOLÍVAR - ALCALDÍA MUNICIPIO DE EL GUAMO,890702015-2,213248011,13,Contratación Mínima Cuantía,Liquidado,Ninguna,3.0,Estatuto General de Contratación,72000000,"Servicios de Edificación, Construcción de Inst...",¿MANTENIMIENTO DE LA INFRAESTRUCTURA DEL ADUL...,Obra,El Guamo,El Guamo,Bolívar - El Guamo,11/28/2012,12-13-1295848,AMEG-072-2012,184,15713124,F,[F] Servicios,0,No Definido,0,No Definido,1219669,Cédula de Ciudadanía,73377382,JESUS DAVID DURAN BARRETO,Bolívar,Cédula de Ciudadanía,73377382,JESUS DAVID DURAN BARRETO,12/04/2012,12/06/2012,15,D,0,0,12/21/2012,No registra,15713124,0,15713124,MANTENIMIENTO DE LA INFRAESTRUCTURA DEL ADULT...,0,No definido,-1,No definido,No definido,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,El Guamo,Bolívar,0
1,12-13-1315899-1303132,2012,2012.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,TOLIMA - ALCALDÍA MUNICIPIO DE ROVIRA,800095983-4,273624011,13,Contratación Mínima Cuantía,Celebrado,Ninguna,3.0,Estatuto General de Contratación,30000000,"Componentes y Suministros para Estructuras, Ed...",MANO DE OBRA PARA LA CONSTRUCCIÓN DE OBRAS DE ...,Obra,Rovira,Rovira,Tolima - Rovira,12/06/2012,12-13-1315899,68-2012,180,15800000,D,[D] Componentes y Suministros,0,No Definido,0,No Definido,1303132,Cédula de Ciudadanía,14397931,OSCAR TOBIAS LOZANO,Tolima,Cédula de Ciudadanía,14397931,OSCAR TOBIAS LOZANO,12/17/2012,12/21/2012,15,D,0,0,01/05/2013,No registra,15800000,0,15800000,MANO DE OBRA PARA LA CONSTRUCCIÓN DE OBRAS DE ...,0,No definido,-1,No definido,No definido,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,Rovira,Tolima,0
2,12-4-1269302-1187119,2012,2012.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2,ANTIOQUIA - EMPRESA DE DESARROLLO URBANO EDU -...,900272057,268001703,4,Régimen Especial,Celebrado,Ninguna,2.0,Régimen Especial,72000000,"Servicios de Edificación, Construcción de Inst...",CONSTRUCCIÓN DE OBRAS DE PAVIMENTO URBANISMO ...,Obra,No definido,No definido,Antioquia - Medellín,11/19/2012,12-4-1269302,ILC 15 DE 2012,447,378209812,F,[F] Servicios,0,No Definido,0,No Definido,1187119,Nit de Persona Jurídica,811.035.837-0,ORGANIZACIÓN DE INGENIERÍA Y TECNOLOGÍA COLOMB...,Antioquia,Cédula de Ciudadanía,71.789.730,JUAN MIGUEL GARZÓN IRAL,1

In [ ]:
secop_obra['Tipo de Contrato'].value_counts()

Obra    226154
Name: Tipo de Contrato, dtype: int64

In [ ]:
secop_obra_sancionados = secop_obra.loc[secop_obra['UID'].isin(secop_i_match.loc[secop_i_match['Match'] == 'si']['UID'].unique())]

In [ ]:
secop_obra_sancionados

,UID,Anno Cargue SECOP,Anno Firma del Contrato,Nivel Entidad,Orden Entidad,Nombre de la Entidad,NIT de la Entidad,Código de la Entidad,ID Tipo de Proceso,Tipo de Proceso,Estado del Proceso,Causal de Otras Formas de Contratacion Directa,ID Regimen de Contratacion,Regimen de Contratacion,ID Objeto a Contratar,Objeto a Contratar,Detalle del Objeto a Contratar,Tipo de Contrato,Municipio Obtencion,Municipio Entrega,Municipios Ejecucion,Fecha de Cargue en el SECOP,Numero de Constancia,Numero de Proceso,Numero del Contrato,Cuantia Proceso,ID Grupo,Nombre Grupo,ID Familia,Nombre Familia,ID Clase,Nombre Clase,ID Ajudicacion,Tipo Identifi del Contratista,Identificacion del Contratista,Nom Raz Social Contratista,Dpto y Muni Contratista,Tipo Doc Representante Legal,Identific del Represen Legal,Nombre del Represen Legal,Fecha de Firma del Contrato,Fecha Ini Ejec Contrato,Plazo de Ejec del Contrato,Rango de Ejec del Contrato,Tiempo Adiciones en Dias,Tiempo Adiciones en Meses,Fecha Fin Ejec Contrato,Compromiso Presupuestal,Cuantia Contrato,Valor Total de Adiciones,Valor Contrato con Adiciones,Objeto del Contrato a la Firma,ID Origen de los Recursos,Origen de los Recursos,Codigo BPIN,Proponentes Seleccionados,Calificacion Definitiva,ID Sub Unidad Ejecutora,Nombre Sub Unidad Ejecutora,Ruta Proceso en SECOP I,Moneda,EsPostConflicto,Marcacion Adiciones,Posicion Rubro,Nombre Rubro,Valor Rubro,Sexo RepLegal Entidad,Pilar Acuerdo Paz,Punto Acuerdo Paz,Municipio Entidad,Departamento Entidad,Multa
51772,14-1-121427-2889616,2014,2014.0,TERRITORIAL,DISTRITO CAPITAL,BOGOTÁ D.C. - IDU,No registra,1002015,1,Licitación Pública,Liquidado,Ninguna,3.0,Estatuto General de Contratación,95000000,"Terrenos, Edificios, Estructuras y Vías",OBRAS Y ACTIVIDADES PARA LA CONSERVACIÓN DE LA...,Obra,Bogotá D.C.,Bogotá D.C.,Bogotá D.C. - Bogotá D.C.,07/02/2014,14-1-121427,IDU-LP-SGI-017-2014,IDU-1686-2014,61760766948,G,"[G] Terrenos, Edificios, Estructuras y vías",0,No Definido,0,No Definido,2889616,Nit de Persona Jurídica,9007835993,CONSORCIO MALLA VIAL 017,Bogotá D.C.,Cédula de Ciudadanía,91454569,carlos augusto ramirez acuña,10/30/2014,11/25/2014,300,D,90,0,02/23/2015,No registra,8875752307,4000000000,12875752307,OBRAS Y ACTIVIDADES PARA LA CONSERVACIÓN DE LA...,0,No definido,-1,GRUPO 1: CONSORCIO MALLA VIAL 17 GRUPO 10: UNI...,GRUPO 1: 8 875 752 307 GRUPO 10: 7.351.564.894...,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,Bogotá D.C.,Bogotá D.C.,1
52918,14-11-2728702-2702030,2014,2014.0,TERRITORIAL,DISTRITO CAPITAL,BOLÍVAR - ALCALDÍA DEL DISTRITO TURÍSTICO Y CU...,No registra,213001021,11,Selección Abreviada de Menor Cuantía (Ley 1150...,Celebrado,Ninguna,3.0,Estatuto General de Contratación,72000000,"Servicios de Edificación, Construcción de Inst...",CONSTRUCCIÓN DE CINCO AULAS BATERÍA SANITARIA...,Obra,Cartagena,Cartagena,Bolívar - Cartagena,07/02/2014,14-11-2728702,SA-MC-DDLP-UAC-018-2014,220-221-222-2014,3575386000,F,[F] Servicios,0,No Definido,0,No Definido,2702030,Nit de Persona Jurídica,8000935461,SEVING SA,Bolívar,Cédula de Ciudadanía,73088927,LUIS CARLOS PADILLA CAMPO,08/22/2014,08/26/2014,5,M,0,0,01/26/2015,No registra,3575386000,0,3575386000,CONSTRUCCIÓN DE CINCO AULAS BATERÍA SANITARIA...,0,No definido,-1,SERVING S.A,SERVING S.A,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,Cartagena,Bolívar,1
53131,14-1-119009-2704598,2014,2014.0,TERRITORIAL,DISTRITO CAPITAL,BOGOTÁ D.C. - IDU,No registra,1002015,1,Licitación Pública,Celebrado,Ninguna,3.0,Estatuto General de Contratación,95000000,"Terrenos, Edificios, Estructuras y Vías",EJECUCION A MONTO AGOTABLE DE DIAGNOSTICO OB...,Obra,Bogotá D.C.,Bogotá D.C.,Bogotá D.C. - Bogotá D.C.,05/26/2014,14-1-119009,IDU-LP-SGI-008-2014,IDU-795-2014,11146050311,G,"[G] Terrenos, Edificios, Estructuras y vías",0,No Definido,0,No Definido,2

In [ ]:
secop_obra['Multa'] = np.int16(secop_obra['UID'].isin(secop_obra_sancionados['UID']))

In [ ]:
secop_obra

,UID,Anno Cargue SECOP,Anno Firma del Contrato,Nivel Entidad,Orden Entidad,Nombre de la Entidad,NIT de la Entidad,Código de la Entidad,ID Tipo de Proceso,Tipo de Proceso,Estado del Proceso,Causal de Otras Formas de Contratacion Directa,ID Regimen de Contratacion,Regimen de Contratacion,ID Objeto a Contratar,Objeto a Contratar,Detalle del Objeto a Contratar,Tipo de Contrato,Municipio Obtencion,Municipio Entrega,Municipios Ejecucion,Fecha de Cargue en el SECOP,Numero de Constancia,Numero de Proceso,Numero del Contrato,Cuantia Proceso,ID Grupo,Nombre Grupo,ID Familia,Nombre Familia,ID Clase,Nombre Clase,ID Ajudicacion,Tipo Identifi del Contratista,Identificacion del Contratista,Nom Raz Social Contratista,Dpto y Muni Contratista,Tipo Doc Representante Legal,Identific del Represen Legal,Nombre del Represen Legal,Fecha de Firma del Contrato,Fecha Ini Ejec Contrato,Plazo de Ejec del Contrato,Rango de Ejec del Contrato,Tiempo Adiciones en Dias,Tiempo Adiciones en Meses,Fecha Fin Ejec Contrato,Compromiso Presupuestal,Cuantia Contrato,Valor Total de Adiciones,Valor Contrato con Adiciones,Objeto del Contrato a la Firma,ID Origen de los Recursos,Origen de los Recursos,Codigo BPIN,Proponentes Seleccionados,Calificacion Definitiva,ID Sub Unidad Ejecutora,Nombre Sub Unidad Ejecutora,Ruta Proceso en SECOP I,Moneda,EsPostConflicto,Marcacion Adiciones,Posicion Rubro,Nombre Rubro,Valor Rubro,Sexo RepLegal Entidad,Pilar Acuerdo Paz,Punto Acuerdo Paz,Municipio Entidad,Departamento Entidad,Multa
0,12-13-1295848-1219669,2012,2012.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,BOLÍVAR - ALCALDÍA MUNICIPIO DE EL GUAMO,890702015-2,213248011,13,Contratación Mínima Cuantía,Liquidado,Ninguna,3.0,Estatuto General de Contratación,72000000,"Servicios de Edificación, Construcción de Inst...",¿MANTENIMIENTO DE LA INFRAESTRUCTURA DEL ADUL...,Obra,El Guamo,El Guamo,Bolívar - El Guamo,11/28/2012,12-13-1295848,AMEG-072-2012,184,15713124,F,[F] Servicios,0,No Definido,0,No Definido,1219669,Cédula de Ciudadanía,73377382,JESUS DAVID DURAN BARRETO,Bolívar,Cédula de Ciudadanía,73377382,JESUS DAVID DURAN BARRETO,12/04/2012,12/06/2012,15,D,0,0,12/21/2012,No registra,15713124,0,15713124,MANTENIMIENTO DE LA INFRAESTRUCTURA DEL ADULT...,0,No definido,-1,No definido,No definido,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,El Guamo,Bolívar,0
1,12-13-1315899-1303132,2012,2012.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,TOLIMA - ALCALDÍA MUNICIPIO DE ROVIRA,800095983-4,273624011,13,Contratación Mínima Cuantía,Celebrado,Ninguna,3.0,Estatuto General de Contratación,30000000,"Componentes y Suministros para Estructuras, Ed...",MANO DE OBRA PARA LA CONSTRUCCIÓN DE OBRAS DE ...,Obra,Rovira,Rovira,Tolima - Rovira,12/06/2012,12-13-1315899,68-2012,180,15800000,D,[D] Componentes y Suministros,0,No Definido,0,No Definido,1303132,Cédula de Ciudadanía,14397931,OSCAR TOBIAS LOZANO,Tolima,Cédula de Ciudadanía,14397931,OSCAR TOBIAS LOZANO,12/17/2012,12/21/2012,15,D,0,0,01/05/2013,No registra,15800000,0,15800000,MANO DE OBRA PARA LA CONSTRUCCIÓN DE OBRAS DE ...,0,No definido,-1,No definido,No definido,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,Rovira,Tolima,0
2,12-4-1269302-1187119,2012,2012.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2,ANTIOQUIA - EMPRESA DE DESARROLLO URBANO EDU -...,900272057,268001703,4,Régimen Especial,Celebrado,Ninguna,2.0,Régimen Especial,72000000,"Servicios de Edificación, Construcción de Inst...",CONSTRUCCIÓN DE OBRAS DE PAVIMENTO URBANISMO ...,Obra,No definido,No definido,Antioquia - Medellín,11/19/2012,12-4-1269302,ILC 15 DE 2012,447,378209812,F,[F] Servicios,0,No Definido,0,No Definido,1187119,Nit de Persona Jurídica,811.035.837-0,ORGANIZACIÓN DE INGENIERÍA Y TECNOLOGÍA COLOMB...,Antioquia,Cédula de Ciudadanía,71.789.730,JUAN MIGUEL GARZÓN IRAL,1

In [ ]:
(
  secop_obra
 .groupby(['Tipo de Contrato','Multa'])
 .size()
 .unstack('Multa')
 .fillna(0)
)

Multa,0,1
Tipo de Contrato,,
Obra,226089,65


In [ ]:
secop_obra.groupby(['Estado del Proceso','Multa']).size().unstack('Multa').fillna(0)

Multa,0,1
Estado del Proceso,,
Adjudicado,4.0,0.0
Celebrado,125449.0,38.0
Convocado,17.0,0.0
Liquidado,99744.0,27.0
Terminado sin Liquidar,875.0,0.0


In [ ]:
secop_obra.groupby(['Nivel Entidad','Multa']).size().unstack('Multa').fillna(0)

Multa,0,1
Nivel Entidad,,
NACIONAL,22714.0,9.0
No Definida,40.0,0.0
TERRITORIAL,203335.0,56.0


In [ ]:
secop_obra.groupby(['Orden Entidad','Multa']).size().unstack('Multa').fillna(0)

Multa,0,1
Orden Entidad,,
AREA METROPOLITANA,1.0,0.0
DISTRITO CAPITAL,9990.0,27.0
NACIONAL CENTRALIZADO,14531.0,8.0
NACIONAL DESCENTRALIZADO,10130.0,1.0
No Definido,40.0,0.0
TERRITORIAL DEPARTAMENTAL CENTRALIZADO,10076.0,7.0
TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,25282.0,14.0
TERRITORIAL DISTRITAL MUNICIPAL NIVEL 1,7582.0,1.0
TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2,17895.0,1.0


In [ ]:
secop_obra.groupby(['Tipo de Contrato','Multa']).size().unstack('Multa').fillna(0)

Multa,0,1
Tipo de Contrato,,
Obra,226089,65


In [ ]:
#secop_ii = pd.read_csv('secop_ii_contratos_5_9_22.gz')

In [ ]:
#secop_ii.head()

In [ ]:
#secop_ii_procesos = pd.read_csv('secop_ii_procesos_5_9_22.gz')

In [ ]:
#secop_ii['Modalidad de Contratacion'].value_counts()

In [ ]:
#modalidades_secop_ii = ['Selección Abreviada de Menor Cuantía',
#                        'Selección abreviada subasta inversa',
#                        'Licitación pública',
#                        'Licitación pública Obra Publica',
#                        'Licitación Pública Acuerdo Marco de Precios',
#                        'Seleccion Abreviada Menor Cuantia Sin Manifestacion Interes']

#secop_ii = secop_ii.loc[secop_ii['Modalidad de Contratacion'].isin(modalidades_secop_ii)]

In [ ]:
#import fuzzymatcher

#secop_ii_merge = secop_ii[['ID Contrato','Nombre Entidad','Referencia del Contrato','Proveedor Adjudicado']].copy()

#left_on = ["Nombre Entidad", "Nombre Contratista", "Numero de Contrato"]
#right_on = ['Nombre Entidad','Proveedor Adjudicado','Referencia del Contrato']

#matched_results = fuzzymatcher.fuzzy_left_join(multas[left_on],
#                                               secop_ii_merge,
#                                               left_on,
#                                              right_on)

In [ ]:
#matched_results.loc[matched_results.best_match_score > 0].sort_values(by='best_match_score', ascending=False).to_excel('secop_ii_match_multas.xlsx', index=False)

## Tabla Maestra

Algunas ideas de variables:
- Experiencia general del contratista al momento de firmar el contrato
- Experiencia específica del contratista al momento de firmar el contrato
- Diversidad de experiencia (Calcular un indice para verificar que tanto varian 
- Multas y sanciones previas
- Porcentaje de anticipo
- Indicar si es consorcio

In [ ]:
secop_obra

,UID,Anno Cargue SECOP,Anno Firma del Contrato,Nivel Entidad,Orden Entidad,Nombre de la Entidad,NIT de la Entidad,Código de la Entidad,ID Tipo de Proceso,Tipo de Proceso,Estado del Proceso,Causal de Otras Formas de Contratacion Directa,ID Regimen de Contratacion,Regimen de Contratacion,ID Objeto a Contratar,Objeto a Contratar,Detalle del Objeto a Contratar,Tipo de Contrato,Municipio Obtencion,Municipio Entrega,Municipios Ejecucion,Fecha de Cargue en el SECOP,Numero de Constancia,Numero de Proceso,Numero del Contrato,Cuantia Proceso,ID Grupo,Nombre Grupo,ID Familia,Nombre Familia,ID Clase,Nombre Clase,ID Ajudicacion,Tipo Identifi del Contratista,Identificacion del Contratista,Nom Raz Social Contratista,Dpto y Muni Contratista,Tipo Doc Representante Legal,Identific del Represen Legal,Nombre del Represen Legal,Fecha de Firma del Contrato,Fecha Ini Ejec Contrato,Plazo de Ejec del Contrato,Rango de Ejec del Contrato,Tiempo Adiciones en Dias,Tiempo Adiciones en Meses,Fecha Fin Ejec Contrato,Compromiso Presupuestal,Cuantia Contrato,Valor Total de Adiciones,Valor Contrato con Adiciones,Objeto del Contrato a la Firma,ID Origen de los Recursos,Origen de los Recursos,Codigo BPIN,Proponentes Seleccionados,Calificacion Definitiva,ID Sub Unidad Ejecutora,Nombre Sub Unidad Ejecutora,Ruta Proceso en SECOP I,Moneda,EsPostConflicto,Marcacion Adiciones,Posicion Rubro,Nombre Rubro,Valor Rubro,Sexo RepLegal Entidad,Pilar Acuerdo Paz,Punto Acuerdo Paz,Municipio Entidad,Departamento Entidad,Multa
0,12-13-1295848-1219669,2012,2012.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,BOLÍVAR - ALCALDÍA MUNICIPIO DE EL GUAMO,890702015-2,213248011,13,Contratación Mínima Cuantía,Liquidado,Ninguna,3.0,Estatuto General de Contratación,72000000,"Servicios de Edificación, Construcción de Inst...",¿MANTENIMIENTO DE LA INFRAESTRUCTURA DEL ADUL...,Obra,El Guamo,El Guamo,Bolívar - El Guamo,11/28/2012,12-13-1295848,AMEG-072-2012,184,15713124,F,[F] Servicios,0,No Definido,0,No Definido,1219669,Cédula de Ciudadanía,73377382,JESUS DAVID DURAN BARRETO,Bolívar,Cédula de Ciudadanía,73377382,JESUS DAVID DURAN BARRETO,12/04/2012,12/06/2012,15,D,0,0,12/21/2012,No registra,15713124,0,15713124,MANTENIMIENTO DE LA INFRAESTRUCTURA DEL ADULT...,0,No definido,-1,No definido,No definido,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,El Guamo,Bolívar,0
1,12-13-1315899-1303132,2012,2012.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 6,TOLIMA - ALCALDÍA MUNICIPIO DE ROVIRA,800095983-4,273624011,13,Contratación Mínima Cuantía,Celebrado,Ninguna,3.0,Estatuto General de Contratación,30000000,"Componentes y Suministros para Estructuras, Ed...",MANO DE OBRA PARA LA CONSTRUCCIÓN DE OBRAS DE ...,Obra,Rovira,Rovira,Tolima - Rovira,12/06/2012,12-13-1315899,68-2012,180,15800000,D,[D] Componentes y Suministros,0,No Definido,0,No Definido,1303132,Cédula de Ciudadanía,14397931,OSCAR TOBIAS LOZANO,Tolima,Cédula de Ciudadanía,14397931,OSCAR TOBIAS LOZANO,12/17/2012,12/21/2012,15,D,0,0,01/05/2013,No registra,15800000,0,15800000,MANO DE OBRA PARA LA CONSTRUCCIÓN DE OBRAS DE ...,0,No definido,-1,No definido,No definido,0,No definida,https://www.contratos.gov.co/consultas/detalle...,Pesos (COP),No Definido,No,No Definido,No Definido,0,ND,No Definido,No Definido,Rovira,Tolima,0
2,12-4-1269302-1187119,2012,2012.0,TERRITORIAL,TERRITORIAL DISTRITAL MUNICIPAL NIVEL 2,ANTIOQUIA - EMPRESA DE DESARROLLO URBANO EDU -...,900272057,268001703,4,Régimen Especial,Celebrado,Ninguna,2.0,Régimen Especial,72000000,"Servicios de Edificación, Construcción de Inst...",CONSTRUCCIÓN DE OBRAS DE PAVIMENTO URBANISMO ...,Obra,No definido,No definido,Antioquia - Medellín,11/19/2012,12-4-1269302,ILC 15 DE 2012,447,378209812,F,[F] Servicios,0,No Definido,0,No Definido,1187119,Nit de Persona Jurídica,811.035.837-0,ORGANIZACIÓN DE INGENIERÍA Y TECNOLOGÍA COLOMB...,Antioquia,Cédula de Ciudadanía,71.789.730,JUAN MIGUEL GARZÓN IRAL,1

In [ ]:
# Crear variable con el rango de la cuantia
secop_obra['Rango Cuantia'] = pd.cut(secop_obra['Cuantia Contrato'], 
                                  bins=[0,1e6,1e7,1e8,1e9,1e10,1e11,1e12,1e13],
                                  labels=['Menos de 1 millon','Entre 1 y 10 millones','Entre 10 y 100 millones','Entre 100 y 1000 millones', 'Entre 1000 y 10000 millones','Entre 10000 y 100000 millones','Entre 100000 millones y 1 billon', 'mas de 1 billon'])

In [ ]:
secop_obra.groupby(['Rango Cuantia']).size()

Rango Cuantia
Menos de 1 millon                     2554
Entre 1 y 10 millones                38072
Entre 10 y 100 millones             108218
Entre 100 y 1000 millones            57844
Entre 1000 y 10000 millones          17118
Entre 10000 y 100000 millones         1912
Entre 100000 millones y 1 billon       125
mas de 1 billon                          1
dtype: int64

### Filtrado inicial

El propósito de este filtro es quitar contratos con las siguientes características:

- Estado del Proceso: Borrador, Convocado, Descartado
- Tipo De Contrato: Acuerdo Marco, Agregación de Demanda, Arrendamiento, Comodato, Crédito, Fiducia, Otro Tipo de Contrato
- Razon social del contratista = No Definido


In [ ]:
(
    secop_obra
    .loc[secop_obra['Tipo de Contrato'] == 'Obra']
    .groupby(['Objeto a Contratar','Multa'], observed=True)
    .size()
    .sort_values(ascending=False)
    #.head(10)
    #.plot(kind='barh')
    .unstack('Multa')
    .fillna(0)
 )

Multa,0,1
Objeto a Contratar,,
"Alimentos, Bebidas y Tabaco",33.0,0.0
"Artículos Domésticos, Suministros y Productos Electrónicos de Consumo",35.0,0.0
Componentes y Equipos para Distribución y Sistemas de Acondicionamiento,319.0,0.0
Componentes y Suministros Electrónicos,365.0,0.0
Componentes y Suministros de Manufactura,327.0,0.0
"Componentes y Suministros para Estructuras, Edificación, Construcción y Obras Civiles",13361.0,3.0
"Componentes, Accesorios y Suministros de Sistemas Eléctricos e Iluminación",1473.0,0.0
Difusión de Tecnologías de Información y Telecomunicaciones,178.0,0.0
"Equipo Médico, Accesorios y Suministros",33.0,0.0


In [ ]:
# Filtrar por estado del proceso
estado_proceso_del = ['Borrador', 'Convocado', 'Descartado']
secop_obra = secop_obra.loc[-secop_obra['Estado del Proceso'].isin(estado_proceso_del)]

# Filtrar por tipo de contrato
#tipo_contrato_del = ['Acuerdo Marco', 'Agregación de Demanda', 'Arrendamiento', 'Comodato', 'Crédito', 'Fiducia', 'Otro Tipo de Contrato', 'Concesión', 'Interventoría', 'Consultoría']
#secop_i = secop_i.loc[-secop_i['Tipo de Contrato'].isin(tipo_contrato_del)]

# Filtrar contratos de mas de 100 millones
#secop_i = secop_i.loc[secop_i['Cuantia Contrato'] > 1e8]

In [ ]:
secop_obra.shape

(226137, 73)

In [ ]:
secop_obra.loc[(secop_obra['Tipo de Contrato'] == 'Obra')].to_csv('obra/secop_i_obra_sanciones_all.csv')